In [12]:
import numpy as np
from scipy import interpolate
import time
import math
import matplotlib.pyplot as plt
from misc.golden_section_search import Search

In [13]:
#calibration parameters
alpha = 0.36
tfp = 1
delta = 0.1
beta = 0.95
r = 0.045
tau = 0.15
gamma = 2
eta = 2
psi = 1 #what did berg have

#model
years_retired = 6          # years during retired 
years_working = 36           # years of working time/
years_childhood = 18


#numerical parameters
kmin = 0            # lower limit capital
kmax = 10           # upper limit of capital
lmin = 0
lmax = 1
hmin = 1
hmax = 10
zmin = 1
zmax = 10 
na = 200            # number of grid points over capital in [kmin,kmax]
k_grid = np.linspace(kmin, kmax, na)   # asset grid 
n_grid = np.linspace(lmin, lmax, na)
h_grid = np.linspace(hmin, hmax, na)
z_grid = np.linspace(zmin, zmax, na)

keps = (k_grid[1]-k_grid[0])/na     #  test for corner solution
neps = (k_grid[1]-k_grid[0])/na 
heps = (k_grid[1]-k_grid[0])/na 
zeps = (k_grid[1]-k_grid[0])/na 


# psi = 0.001         # parameter of utility function */
phi = 0.8           # updating of the aggregate capital stock K in outer loop
tolerance = 0.0001        # percentage deviation of final solution 
tolerance_gs = 1e-10        # tolerance for golden section search 
neg = -1e10         # initial value for value function 
max_iters_k = 30             # maximum number of iteration over K
small = 0.001 # small so 0 consumption will still yield positive utility

In [14]:
# initialization of aggregate variables
# Step 1 in Algorithm 9.1.1 in Heer/Maussner, DSGE Modeling (Springer) 
zbar = 1
lbar = 0.2      # aggregate labor L
hbar = 1
kbar = ((r + delta)/(alpha * tfp * zbar ** psi)) ** (1/(alpha - 1)) * (hbar * lbar) # aggregate capital K
kold = 100      # capital stock in previous iteration of K, initialization
lold = 2        # labor in previous iteration of N, initialization
zold = 10
hold = 10
aggregates_iteration_array = np.zeros((max_iters_k,4))   # saves (K,N) in each outer iteration

print(kbar)

0.8281617148928373


In [15]:
wage_rate = lambda h, k, l, z: h * (1-alpha) * tfp * z**psi * k**alpha * (h * l)**(-alpha)
capital_rate = lambda h, k, l, z: alpha *  tfp * z**psi * k**(alpha-1) * (h * l)**(1 - alpha)
interest_rate = lambda h, k, l, z: alpha *  tfp * z**psi * k**(alpha-1) * (h * l)**(1 - alpha) - delta
transfers = lambda c, ie, iz, tau, h, l, k: tau*c - ie - iz + tau*h*l + tau*interest_rate*k #assuming balanced budget, remove borrowing, time subscripts wrong

In [16]:
w = wage_rate(hbar, kbar, lbar, zbar)
r = interest_rate(hbar, kbar, lbar, zbar)

print(w, r)

1.067408432528546 0.04500000000000001


In [17]:
# abbreviations
exp = np.e
log = math.log

In [18]:
# utility = lambda c, n: ((c * (1 - n) ** gamma) ** (1 - eta) - 1)/(1 - eta)
def utility(c, l):
  if l == 1:
    return np.log(c+ small) + gamma * np.log(l)
  return (((c + small) * (1 - l) ** gamma) ** (1 - eta) - 1) / (1 - eta)

#bellman retired
# input: next-period capital k_tp1
# output: rhs of Bellman equation, future could make 1 bellman, params diff
def value1(k_tp1):
  c = ((1+r)*k0 - max(0, tau * r * k0) + t - k_tp1) / (1 + tau) #post tax consumption = assets
  if c<=0:
    return neg
  else:
    return utility(c,0) + beta * rvalue(k_tp1)
  
# interpolation of next-period value function
# for the retired at wealth k    
def rvalue(k):
    return vr_polate(k)

# Bellman equation of the worker
# input: next-period wealth a'
# output: rhs of Bellman equation
def value2(k_tp1):
    # optimal labor supply implied by first-order condition
    l = 1/(1+gamma) * (1- gamma/((1-tau)*w) * (small+(1+r)*k0-k_tp1) ) #NEED TO FIX
    
    if l<0: # corner solution 0<=n<=1
        l = 0
    elif l>=1: # corner solution
        l = 1
    
    c = ((1+r)*k0 - max(0, tau * r * k0) + t - k_tp1) / (1 + tau)
    if c<=0:
        return neg
    
    return utility(c,1-l) + b*vw_polate(x)

In [19]:
iteration = -1
crit = tolerance + 1

while iteration < max_iters_k - 1  and crit > tolerance:
  iteration += 1
  print(f"iteration: {iteration}")

  #get percentage deviation of solutions in iteration q and q-1
  crit_k = abs((kbar-kold)/kbar) 
  crit_l = abs((lbar-lold)/lbar)
  crit_h = abs((hbar-hold)/hbar)
  crit_z = abs((zbar-zold)/zbar)
  
  print(f"k deviations: {round(crit_k, 2)}")
  print(f"l deviations: {round(crit_l, 2)}")
  print(f"h deviations: {round(crit_h, 2)}")
  print(f"z deviations: {round(crit_z, 2)}")


  w = wage_rate(hbar, kbar, lbar, zbar)
  r = interest_rate(hbar, kbar, lbar, zbar)

  kold, lold, hold, zold = kbar, lbar, lold, zold
  aggregates_iteration_array[iteration, 0] = kbar
  aggregates_iteration_array[iteration, 1] = lbar
  aggregates_iteration_array[iteration, 2] = hbar
  aggregates_iteration_array[iteration, 3] = zbar

  #retired value function
  value_function_retired = np.zeros((na, years_retired))
  optimal_capital_retired = np.zeros((na, years_retired))
  optimal_labor_retired = np.zeros((na, years_retired))
  optimal_consumption_retired = np.zeros((na, years_retired))

  #figure out how to get govt transfers t

  #last period of life, need to figure out way to store k prev
  for i in range(na):
    c = ((1+r)*k_grid[i] - max(0, tau * r * k_grid[i]) + t) / (1 + tau)
    value_function_retired[i, years_retired - 1] = utility(c,1)
    optimal_consumption_retired[i, years_retired - 1] = c


  #worker value function    # workers' value function 
  value_function_worker = np.zeros((na, years_working))
  optimal_capital_worker = np.ones((na, years_working))
  optimal_consumption_worker = np.zeros((na, years_working))
  optimal_labor_worker = np.zeros((na, years_working))

  #computer retiree policy function
  for i in range(years_retired - 1, 0, -1):
    print("*" * 30)
    print(f"iteration: {iteration}, age: {years_working + i}")
    print(f"k: {kbar}, l: {lbar}, h: {hbar}, z: {zbar}")
    print("*" * 30)

    value_function_retired_polate = interpolate.interp1d(k_grid, value_function_retired[:,i], fill_value='extrapolate')
    # value_function_retired_polate = interpolate.interp1d(k_grid, value_function_retired[:,i], fill_value='extrapolate', kind="cubic")

    m0 = 0
             
    for j in range(na): # asset holding at age i
      k0 = k_grid[j]
      # triple ax, bx, cx: [ax,bx] bracket the maximum of Bellman eq.
      ax = 0
      bx = -1
      cx = -2
      v0 = neg
      m = max(-1,m0-2)
      # locate ax <= a' <= cx that bracket the maximum
      while ax>bx or bx>cx:
        m = m+1
        v1 = value1(k_grid[m])
                
        if v1>v0:
          if m==0: # new value at lower bound a[0]
            ax=k_grid[m] 
            bx=k_grid[m]
          else:
            bx=k_grid[m] 
            ax=k_grid[m-1]
                    
            v0 = v1
            m0=m   # monotonocity of the value function 
                    
        else:
          cx=k_grid[m]
                
        if m==na-1:
          ax = k_grid[m-1]
          bx = k_grid[m] 
          cx = k_grid[m] 

      if ax==bx:  # corner solution: a'=0?
        if value1(ax)>value1(keps):
          optimal_capital_worker[j,i-1]=0
        else:
          optimal_capital_worker[j,i-1] = Search(value1,ax,keps,cx,tolerance_gs).find_max()
                
      elif bx==cx:  # corner solution: a'=a[na-1]=kmax?
        if value1(k_grid[na-1])>value1(k_grid[na-1]-keps):
          keps[j,i-1] = k_grid[na-1]
        else:
          optimal_capital_worker[j,i-1] = Search(value1,k_grid[na-2],kmax-keps,kmax,tolerance_gs).find_max() 
      else:
        optimal_capital_worker[j,i-1] = Search(value1,ax,bx,cx,tolerance_gs).find_max()
            

      k1 = optimal_consumption_retired[j,i-1]
      value_function_retired[j,i-1] = value1(optimal_capital_worker[j,i-1])
      optimal_consumption_retired[j,i-1] = (1+r)*k_grid[j]+t-k1


iteration: 0
k deviations: 119.75
l deviations: 9.0
h deviations: 9.0
z deviations: 9.0


NameError: name 't' is not defined